In [ ]:
import os
import pandas as pd
import json
import numpy as np


In [ ]:
prj_root = os.getcwd()
dataset_folderpath= os.path.join(prj_root,'mitre_nav_reports')
ttp_list_corpus=[]

In [ ]:
for root, dirs, files in os.walk(dataset_folderpath):
    for filename in files:
        abs_filepath = os.path.join(root,filename)
        # print(abs_filepath)
        with open(abs_filepath,'r') as f:
            data = data = json.load(f)

        df = pd.DataFrame(data['techniques'])
        # df = df[df['score'].notna()]
        df = df.loc[df['score'].isin([1.0])]
        ttp_list= df['techniqueID'].tolist()
        for ttp in ttp_list:
            ttp_list_corpus.append(ttp)
        
        # ttp_list_corpus.append(' '.join(ttp_list))
    
#     break

In [ ]:
# src:https://github.com/kavgan/nlp-in-practice/blob/master/tf-idf/Keyword%20Extraction%20with%20TF-IDF%20and%20SKlearn.ipynb
from sklearn.feature_extraction.text import CountVectorizer
#instantiate CountVectorizer() 
cv=CountVectorizer(analyzer=lambda d: d.split(' ')) 
word_count_vector=cv.fit_transform(ttp_list_corpus)

In [ ]:
#freq_count of all ttps
freq_count_dict = dict(zip(cv.get_feature_names(),word_count_vector.toarray().sum(axis=0)))

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
# tfidf_transformer.idf_

In [ ]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [ ]:
# you only needs to do this once
feature_names=cv.get_feature_names()

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([' '.join(ttp_list_corpus)]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,len(feature_names))


# for k in keywords:
#     print(k,keywords[k])

In [ ]:
df1 = pd.DataFrame([keywords])
df1 = df1.T
df1.index.name = 'TTP'
df1.columns = ['weight']
df1
# df1.to_csv (r'tf-tdf.csv', index = True, header=True)

In [ ]:
ttp_list_corpus.sort()
len(ttp_list_corpus)

In [ ]:
df3 = pd.DataFrame(ttp_list_corpus,
               columns =['TTP'])
s = df3.value_counts(normalize=True).sort_index()
pd11 = pd.DataFrame(s).reset_index()
pd11.columns = ['TTP', 'weight']
# s= s.sort_values(ascending=False)
pd11.sort_values(by=['weight'],ascending=False, inplace=True)
pd11
# pd11.to_csv('freq_percentage.csv',index=False)

In [ ]:
df4 = df3.groupby(by=["TTP"]).size().reset_index(name='counts')
df4

In [ ]:
# src:https://www.geeksforgeeks.org/data-normalization-with-pandas/
# Using The maximum absolute scaling
# copy the data
df_max_scaled = df4.copy()
# df_max_scaled = df_max_scaled.drop(columns=['TTP'])
df_max_scaled
# apply normalization techniques

df_max_scaled['weight'] = df_max_scaled['counts']  / df_max_scaled['counts'] .abs().max()
      
# view normalized data
df_max_scaled= df_max_scaled.sort_values('weight',ascending=False)
# df_max_scaled.to_csv('maximum_absolute_scaling.csv',index=False)

In [ ]:
# Using The min-max feature scaling
# copy the data
df_min_max_scaled = df4.copy()
  
# apply normalization techniques
df_min_max_scaled['NORM'] = ((df_min_max_scaled['counts']- df_min_max_scaled['counts'].min()) / (df_min_max_scaled['counts'].max() - df_min_max_scaled['counts'].min()))   
  
# view normalized data
df_min_max_scaled=df_min_max_scaled.sort_values('NORM',ascending=False)
# df_min_max_scaled.to_csv(' min_max_scaling.csv',index=False)

In [ ]:
ttp_list_corpus